In [3]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('admission.csv')
df.head()

,admit,gre,gpa,rank
0,0.0,380.0,3.61,3.0
1,1.0,660.0,3.67,3.0
2,1.0,800.0,4.00,1.0
3,1.0,640.0,3.19,4.0
4,0.0,520.0,2.93,4.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    float64
 1   gre     400 non-null    float64
 2   gpa     400 non-null    float64
 3   rank    400 non-null    float64
dtypes: float64(4)
memory usage: 12.6 KB


In [6]:
df = df.dropna()

In [7]:
df[['gre','admit']].groupby('admit').mean()

,gre
admit,
0.0,573.186813
1.0,618.897638


In [10]:
test_gre = pd.cut(x = df['gre'],
                  bins = [200, 450, 550, 620, 800],
                  labels = ['mauvais', 'moyen', 'moyen+', 'bon'])
pd.crosstab(df['admit'], test_gre)

gre,mauvais,moyen,moyen+,bon
admit,,,,
0.0,42,71,73,87
1.0,6,31,33,57


In [11]:
niveaux_notes = pd.cut(x = df['gpa'],
                       bins = [2, 2.8, 3.2, 3.6, 4],
                       labels = ['mauvais', 'moyen', 'moyen+', 'bon'])
pd.crosstab(df['admit'], niveaux_notes, normalize='columns')

gpa,mauvais,moyen,moyen+,bon
admit,,,,
0.0,0.8,0.757282,0.706667,0.565574
1.0,0.2,0.242718,0.293333,0.434426


In [12]:
df = df.join(pd.get_dummies(niveaux_notes, prefix='niveau'))
df = df.join(pd.get_dummies(test_gre,prefix='gre'))

In [14]:
df = df.join(pd.get_dummies(df.rank, prefix='prest'))

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 13 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [16]:
data = df.iloc[:,4:13]
target = df['admit']

In [17]:
data.columns

Index(['niveau_mauvais', 'niveau_moyen', 'niveau_moyen+', 'niveau_bon',
       'gre_mauvais', 'gre_moyen', 'gre_moyen+', 'gre_bon',
       'prest_<bound method NDFrame.rank of      admit    gre   gpa  rank  niveau_mauvais  niveau_moyen  niveau_moyen+  \\n0      0.0  380.0  3.61   3.0           False         False          False   \n1      1.0  660.0  3.67   3.0           False         False          False   \n2      1.0  800.0  4.00   1.0           False         False          False   \n3      1.0  640.0  3.19   4.0           False          True          False   \n4      0.0  520.0  2.93   4.0           False          True          False   \n..     ...    ...   ...   ...             ...           ...            ...   \n395    0.0  620.0  4.00   2.0           False         False          False   \n396    0.0  560.0  3.04   3.0           False          True          False   \n397    0.0  460.0  2.63   2.0            True         False          False   \n398    0.0  700.0  3.65   2.0     

In [52]:
x_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=66) 
